# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
# Load the Stanford Sentiment Treebank dataset
# See: https://huggingface.co/datasets/sst2

# Define the splits we want to load (training and testing)
splits = ["train", "validation"]
# Load the SST-2 dataset splits using a dictionary comprehension.
# 'load_dataset' function fetches the dataset from Hugging Face's dataset repository.
# 'glue' is the broader dataset collection, 'sst2' is the specific dataset for sentiment analysis.
# Iterating over the splits list to load both training and testing sets.
dataset = {split: load_dataset("glue", "sst2", split=split) for split in splits}

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Load tokenizer and tokenize the dataset

In [3]:
#load the tokenzier
tokenizer = AutoTokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
# set EOS (end of sentence) TOKEN as PAD TOKEN
tokenizer.pad_token = tokenizer.eos_token

In [5]:
tokenizer.pad_token

'<|endoftext|>'

In [6]:
def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples.
    :examples:
    
    :returns:
    """
    # convert the text data in a list of tokens using tokenizer, truncating
    # the text to the maximum lenght and pad shorter sequences to a uniform lenght
    # return the result
    return tokenizer(examples['sentence'], padding=True, truncation=True)

In [7]:
# Initialize an empty dictionary to store the tokenized datasets.
tokenized_ds = {}
# Iterate over each data split ('train' and 'test').
for split in splits:
    # Apply the preprocess_function to the dataset corresponding to the current split.
    # The 'map' function applies the preprocess_function to each example in the dataset.
    # 'batched=True' allows processing multiple examples at once for efficiency.
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [8]:
# print a sample of sentence and its tokenization in train subset
print(tokenized_ds["train"][0]['sentence'])
print(tokenized_ds["train"][0]["input_ids"])

hide new secretions from the parental units 
[24717, 649, 3200, 507, 422, 262, 21694, 4991, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]


In [9]:
# print a sample of sentence and its tokenization in validation subset
print(tokenized_ds["validation"][0]['sentence'])
print(tokenized_ds["validation"][0]["input_ids"])

it 's a charming and often affecting journey . 
[270, 705, 82, 257, 23332, 290, 1690, 13891, 7002, 764, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]


In [10]:
# Load the pre-trained model 'gpt-2' for sequence classification.
# This model is designed for tasks like sentiment analysis where each sequence (like a sentence)
# is classified into categories (like positive/negative).
# here, we specify the number of labels (2 for sentiment classification),
# id2label and label2id corresponding to POSITIVE and NEGATIVE labels
model = AutoModelForSequenceClassification.from_pretrained('gpt2',
                                                      num_labels=2,
                                                      id2label={0: "NEGATIVE", 1: "POSITIVE"},
                                                      label2id={"NEGATIVE": 0, "POSITIVE": 1})

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Set the model's pad token id to match the tokenizer's pad token id
model.config.pad_token_id = tokenizer.pad_token_id

In [12]:
# Freeze all the parameters of the base model
# Iterate over all the parameters of the base model.
for param in model.base_model.parameters():
    # freeze the base model disabling the gradient calculations for each parameter
    # in the base model of "gpt2" model
    # Set 'requires_grad' to False to freeze the parameters of the base model.
    # Freezing prevents the weights of these layers from being updated during training.
    param.requires_grad = False

In [13]:
# check model architecture
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [14]:
# The 'model.score' is the classification head that will be trained to adapt 
# the base model for our specific task (sentiment analysis in this case).
model.score

Linear(in_features=768, out_features=2, bias=False)

In [15]:
def compute_metrics(eval_pred):
    """
    Function for compute tha accuracy metric
    :eval_pred: a tuple with predictions and labels
    
    :returns: a dictionary with the mean accuracy
    """
    predictions, labels = eval_pred
    # Convert the predictions to discrete labels by taking the argmax,
    # which is the index of the highest value in the prediction (logits).
    predictions = np.argmax(predictions, axis=1)
    # Calculate and return the accuracy as the mean of the instances where
    # predictions match the true labels.
    return {"accuracy": (predictions == labels).mean()}

In [16]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Initialize the Trainer, a high-level API for training transformer models.
training_args = TrainingArguments(
    output_dir="./model_output", # Directory where the model outputs will be saved.
    learning_rate=2e-5, # Learning rate for the optimizer.
    # Reduce the batch size if you don't have enough memory
    per_device_train_batch_size=16, # Batch size for training per device.
    per_device_eval_batch_size=16, # Batch size for evaluation per device.
    num_train_epochs=1, # Number of training epochs.
    weight_decay=0.01, # Weight decay for regularization.
    evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
    save_strategy="epoch", # Model is saved at the end of each epoch.
    load_best_model_at_end=True, # Load the best model at the end of training.
)

pretrain_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"], # The tokenized training dataset.
    eval_dataset=tokenized_ds["validation"], # The tokenized evaluation dataset.
    tokenizer=tokenizer, # The tokenizer used for encoding the data.
    # Data collator that will dynamically pad the batches during training.
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics, # Function to compute metrics during evaluation.
)

In [17]:
# Evaluate the model on the validation set before fine-tuning
pretrain_results = pretrain_trainer.evaluate()

# Print the evaluation results before fine-tuning
print("Evaluation results before fine-tuning:", pretrain_results)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluation results before fine-tuning: {'eval_loss': 2.5617620944976807, 'eval_accuracy': 0.5091743119266054, 'eval_runtime': 13.049, 'eval_samples_per_second': 66.825, 'eval_steps_per_second': 4.215}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [19]:
from peft import LoraConfig, get_peft_model, TaskType

In [18]:
# Load the pre-trained model 'gpt-2' for sequence classification.
# This model is designed for tasks like sentiment analysis where each sequence (like a sentence)
# is classified into categories (like positive/negative).
# here, we specify the number of labels (2 for sentiment classification),
# id2label and label2id corresponding to POSITIVE and NEGATIVE labels
model = AutoModelForSequenceClassification.from_pretrained('gpt2',
                                                      num_labels=2,
                                                      id2label={0: "NEGATIVE", 1: "POSITIVE"},
                                                      label2id={"NEGATIVE": 0, "POSITIVE": 1})

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Set the model's pad token id to match the tokenizer's pad token id
model.config.pad_token_id = tokenizer.pad_token_id

In [21]:
# Create a PEFT Config for LoRA
config = LoraConfig(
                    r=8, # Rank
                    lora_alpha=32,
                    target_modules=['c_attn', 'c_proj'],
                    lora_dropout=0.1,
                    bias="none",
                    task_type=TaskType.SEQ_CLS
                )

peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 814,080 || all params: 125,253,888 || trainable%: 0.6499438963523432


In [22]:
# Rename 'label' to 'labels' to match the Trainer's expectation
tokenized_ds["train"] = tokenized_ds["train"].map(lambda e: {'labels': e['label']}, batched=True, remove_columns=['label'])
tokenized_ds["validation"] = tokenized_ds["validation"].map(lambda e: {'labels': e['label']}, batched=True, remove_columns=['label'])

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [23]:
tokenized_ds["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_ds["validation"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [24]:
# Initialize the Trainer
trainer = Trainer(
    model=peft_model,  # Make sure to pass the PEFT model here
    args=TrainingArguments(
        output_dir="./lora_model_output",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=2,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_dir='./logs',  # If you want to log metrics and/or losses during training
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=512),
    compute_metrics=compute_metrics,
)

In [25]:
# Start the training process
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.350500,0.282903,0.888761
2,0.325700,0.274430,0.899083


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=4210, training_loss=0.380601579389776, metrics={'train_runtime': 1358.0802, 'train_samples_per_second': 99.183, 'train_steps_per_second': 3.1, 'total_flos': 4436193624975360.0, 'train_loss': 0.380601579389776, 'epoch': 2.0})

In [26]:
# Save fine tuned PEFT model
peft_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [27]:
import torch
from peft import AutoPeftModelForSequenceClassification

NUM_LABELS = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora", num_labels=NUM_LABELS, ignore_mismatched_sizes=True).to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Set the model's pad token id to match the tokenizer's pad token id
lora_model.config.pad_token_id = tokenizer.pad_token_id

In [29]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Initialize the Trainer, a high-level API for training transformer models.
training_args = TrainingArguments(
    output_dir="./data/sentiment_analysis", # Directory where the model outputs will be saved.
    learning_rate=2e-5, # Learning rate for the optimizer.
    # Reduce the batch size if you don't have enough memory
    per_device_train_batch_size=16, # Batch size for training per device.
    per_device_eval_batch_size=16, # Batch size for evaluation per device.
    num_train_epochs=1, # Number of training epochs.
    weight_decay=0.01, # Weight decay for regularization.
    evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
    save_strategy="epoch", # Model is saved at the end of each epoch.
    load_best_model_at_end=True, # Load the best model at the end of training.
)

finetuned_trainer = Trainer(
    model=lora_model,  # The fine-tuned PEFT model.
    args=training_args,# Training arguments, defined above.
    train_dataset=tokenized_ds["train"], # The tokenized training dataset.
    eval_dataset=tokenized_ds["validation"], # The tokenized evaluation dataset.
    tokenizer=tokenizer, # The tokenizer used for encoding the data.
    # Data collator that will dynamically pad the batches during training.
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics, # Function to compute metrics during evaluation.
)

In [30]:
# Evaluate the fine-tuned model on the validation set
finetuned_results = finetuned_trainer.evaluate()

# Print the evaluation results for the fine-tuned model
print("Evaluation results for the fine-tuned model:", finetuned_results)

Evaluation results for the fine-tuned model: {'eval_loss': 0.2744295299053192, 'eval_accuracy': 0.8990825688073395, 'eval_runtime': 3.745, 'eval_samples_per_second': 232.842, 'eval_steps_per_second': 14.686}
